In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from IPython.core.display import HTML
import datetime
import numpy as np
import numpy.ma as ma
import os

In [ ]:
#year = 18; w = 469
year = 19; w = 543

In [ ]:
save = os.getcwd()
save = os.path.join(save,"save")
save = os.path.join(save,str(year))
save = os.path.join(save,'df_rich.pickle')

with open(save, 'rb') as f:
    df = pickle.load(f)

In [ ]:
df = df.rename(columns={"position":"Position"})

In [ ]:
clubdict = dict(zip(df["club"].unique().tolist(),np.arange(20).tolist()))

In [ ]:
clubdict

## Random Guess

In [ ]:
dfl = df.loc[:w]
rnum = np.random.rand((dfl.shape[0]))
dfl.insert(dfl.shape[1],"rand",rnum,True)
goalies = dfl[dfl['Position'] == "Goalkeeper"]
defenders = dfl[dfl['Position'] == "Defender"]
midfielders = dfl[dfl['Position'] == "Midfielder"]
forwards = dfl[dfl['Position'] == "Forward"]

In [ ]:
randomteam = pd.concat([goalies.nlargest(1,"rand"),defenders.nlargest(4,"rand"),midfielders.nlargest(4,"rand"),forwards.nlargest(2,"rand")])

In [ ]:
np.histogram(randomteam["club"].map(clubdict))[0].max()

In [ ]:
np.sum(randomteam['Highest R1 Bid']+0.5)

In [ ]:
np.sum(randomteam['Points'])

In [ ]:
bestrandomlist = []
bestpointslist = []
for m in range(10):
    bestrandom = None
    bestpoints = 0
    for n in range(1000):
        dfl = df.loc[:w]
        rnum = np.random.rand((dfl.shape[0]))
        dfl.insert(dfl.shape[1],"rand",rnum,True)
        goalies = dfl[dfl['Position'] == "Goalkeeper"]
        defenders = dfl[dfl['Position'] == "Defender"]
        midfielders = dfl[dfl['Position'] == "Midfielder"]
        forwards = dfl[dfl['Position'] == "Forward"]
        randomteam = pd.concat([goalies.nlargest(1,"rand"),defenders.nlargest(4,"rand"),midfielders.nlargest(4,"rand"),forwards.nlargest(2,"rand")])
        if np.histogram(randomteam["club"].map(clubdict))[0].max() < 3:
            if np.sum(randomteam['Highest R1 Bid']+0.5) <= 75:
                if np.sum(randomteam['Points']) > bestpoints:
                    bestrandom = randomteam
                    bestpoints = np.sum(randomteam['Points'])
    bestrandomlist.append(bestrandom)
    bestpointslist.append(bestpoints)

In [ ]:
allrands = bestrandomlist[0]
for x in range(1,len(bestrandomlist)):
    allrands = pd.concat([allrands,bestrandomlist[x]])

In [ ]:
allrands["Name"].unique().size

In [ ]:
(max(bestpointslist) - min(bestpointslist))/sum(bestpointslist)*len(bestpointslist)*100

## Dynamic Programming

In [ ]:
items = df[['Name','Highest R1 Bid','Points']]
items.Points = items.Points.astype(int)
items['Highest R1 Bid'] = items['Highest R1 Bid'].astype(int)*2

In [ ]:
def knapsack_pandas(items, limit):
    table = np.zeros((items.shape[0]+1,limit+1),dtype=int)
 
    for j in range(1, items.shape[0] + 1):
        item, wt, val = items.iloc[j-1]
        for w in range(1, limit + 1):
            if wt > w:
                table[j][w] = table[j-1][w]
            else:
                table[j][w] = max(table[j-1][w],
                                  table[j-1][w-wt] + val)
 
    result = pd.DataFrame(columns=['item','weight','value'])
    
    w = limit
    for j in range(len(items), 0, -1):
        was_added = table[j][w] != table[j-1][w]
 
        if was_added:
            item, wt, val = items.iloc[j-1]
            result = result.append(items.iloc[j-1])
            w -= wt
            
    return (result, table) 

In [ ]:
(bagged, table) = knapsack_pandas(items, 75)
display(HTML(bagged.to_html()))

## Linear Programming

In [ ]:
from scipy.optimize import linprog
dfl = df.iloc[0:w]

In [ ]:
points = dfl['Points']
cost = dfl['Highest R1 Bid']+0.5
gk   = dfl['Position'].apply(lambda x: 1 if x == 'Goalkeeper' else 0)
defe = dfl['Position'].apply(lambda x: 1 if x == 'Defender' else 0)
mid  = dfl['Position'].apply(lambda x: 1 if x == 'Midfielder' else 0)
stri = dfl['Position'].apply(lambda x: 1 if x == 'Forward' else 0)
xi   = np.ones(dfl.shape[0])

In [ ]:
A_upperbounds = np.array([cost])
b_upperbounds = np.array([75])
A_equality = np.array([gk, defe, mid, stri, xi])
b_equality = np.array([1, 4, 4, 2, 11])
bounds = [(0, 1) for x in range(dfl.shape[0])]

In [ ]:
solution = linprog(
    c=-points,
    A_ub=A_upperbounds,
    b_ub=b_upperbounds,
    A_eq=A_equality,
    b_eq=b_equality,
    bounds=bounds
)

In [ ]:
dfs = dfl[solution.x>0.001]
dfs

In [ ]:
solution.x[solution.x>0.001]

In [ ]:
dfs[['Name','Highest R1 Bid']]

## Integrer Programming

In [ ]:
from pulp import *
dfi = df.iloc[0:w]

In [ ]:
dfi.club.unique()

In [ ]:
player = [dfi['Name'][i] for i in range(dfi.shape[0])]
point = {dfi['Name'][i]: dfi['Points'][i] for i in range(dfi.shape[0])} 
cost = {dfi['Name'][i]: dfi['Highest R1 Bid'][i] + 0.5 for i in range(dfi.shape[0])}
gk = {dfi['Name'][i]: 1 if dfi['Position'][i] == 'Goalkeeper' else 0 for i in range(dfi.shape[0])}
defe = {dfi['Name'][i]: 1 if dfi['Position'][i] == 'Defender' else 0 for i in range(dfi.shape[0])}
mid = {dfi['Name'][i]: 1 if dfi['Position'][i] == 'Midfielder' else 0 for i in range(dfi.shape[0])}
stri = {dfi['Name'][i]: 1 if dfi['Position'][i] == 'Forward' else 0 for i in range(dfi.shape[0])}
city = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Manchester City' else 0 for i in range(dfi.shape[0])}
pool = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Liverpool' else 0 for i in range(dfi.shape[0])}
chel = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Chelsea' else 0 for i in range(dfi.shape[0])}
arse = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Arsenal' else 0 for i in range(dfi.shape[0])}
tot = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Tottenham' else 0 for i in range(dfi.shape[0])}
utd = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Manchester Utd' else 0 for i in range(dfi.shape[0])}
bmo = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Bournemouth' else 0 for i in range(dfi.shape[0])}
lei = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Leicester City' else 0 for i in range(dfi.shape[0])}
xi = {dfi['Name'][i]: 1 for i in range(dfi.shape[0])}

In [ ]:
prob = LpProblem("Fantasy Football",LpMaximize)
player_vars = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_vars[i] for i in player]), "Total Cost"

# constraint
prob += lpSum([player_vars[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_vars[i] for i in player]) <= 75.0, "Total Cost"
prob += lpSum([gk[i] * player_vars[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defe[i] * player_vars[i] for i in player]) == 4, "Less than 4 DEF"
prob += lpSum([mid[i] * player_vars[i] for i in player]) == 4, "Less than 5 MID"
prob += lpSum([stri[i] * player_vars[i] for i in player]) == 2, "Less than 3 STR"

# solve
status = prob.solve()

In [ ]:
names = []
for v in prob.variables():
    if v.varValue>0:
        nv = v.name.split("_")[1:]
        if len(nv) > 2:
            ln = "-".join(nv[-2:])
            ln2 = " ".join(nv[-2:])
        else:
            ln = nv[-1]
        fn = nv[0]
        if fn == ln:
            names.append(fn)
        else:
            names.append(fn + " " + ln)
            try:
                names.append(fn + " " + ln2)
            except:
                pass
names

In [ ]:
df_select = dfi[dfi.Name.isin(names)]
df_select

In [ ]:
df_select["Bought for"] = df_select['Highest R1 Bid'] + 0.5
df_select[["Name","club","Position","Points","Bought for"]]

In [ ]:
df_select.Points.sum()

In [ ]:
df_select["Bought for"].sum()